<a href="https://colab.research.google.com/github/ljpetkovic/Charcot_KeyBERT_Keyphrase-Vectorizers/blob/main/scripts/KeyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction des mots/phrases-clés avec `keybert` et `keyphrase-vectorizers`
---

# `keybert`
* librairie Python pour extraire des mots/phrases-clés les plus similaires à un document en exploitant les plongements BERT<br>
⚠️ on doit spécifier la longueur des n-grammes à extraire, alors que l'on ne sait pas quelle est la longueur optimale<br>
⚠️ la grammaticalité des phrases n'est pas prise en compte

<p align="right"><a href="https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db">Schopf, 2022</a></p>


In [30]:
# !pip install keybert

import torch # print(torch.__version__)
import os
from google.colab import drive
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

""" Initialiser le modèle de phrase :
en l'occurrence, le modèle dérive
les plongements de phrases sémantiquement signifiants
qui peuvent être comparées en utilisant
la similarité cosinus.
"""
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
kw_model = KeyBERT(model=sentence_model)

# Monter le Google Drive
drive.mount('/content/drive')

# Definir les chemins vers les fichiers d'entrée et de sortie
path = '/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/corpus/'
file_name = 'echantillon_charcot.txt'
file_path = '../output/test_keybert.txt'

""" Extraction des mots-clés
1) Si on divise le texte en tranches
"""
with open(os.path.join(path, file_name), 'r') as myfile, open(os.path.join(path, file_path), 'w') as outfile:
    raw_data = myfile.readlines()
    start = 0
    end = 200 # diminuer le nb de lignes à traiter si la mémoire RAM est épuisée
    while len(raw_data) >= end:
        data = " ".join(raw_data[start:end])
        start = end
        end += 200
        keywords = kw_model.extract_keywords(data, keyphrase_ngram_range=(1, 3), stop_words=None, use_mmr=True, diversity=0.7)
        for k in keywords:
            print(k)
            # Writing each keyword to the output file
            outfile.write(str(k) + '\n')

""" 2) Si on passe le fichier entier """

# with open(os.path.join(path, file_name), 'r') as myfile, open(os.path.join(path, file_path), 'w') as outfile:
#     data = myfile.readlines(10000) # pour s'arrêter après 10 000 premiers caractères
#     data = [line.strip('\n') for line in data]
#     data = ' '.join(data)
#     # print(data)
#     keywords = kw_model.extract_keywords(data, keyphrase_ngram_range=(1, 3), stop_words=None, use_mmr=True, diversity=0.7)
#     for k in keywords:
#         print(k)
#         # Writing each keyword to the output file
#         outfile.write(str(k) + '\n')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
('moëlle épinière 45', 0.3381)
('topographie anatomo pathologique', 0.2841)
('paul auteur date', 0.2187)
('texte explicatif mais', 0.1587)
('du système', -0.1271)
('scientifique les planches', 0.4944)
('chambre photographique horizontale', 0.1779)
('objectif de appareil', 0.1028)
('et nous lui', 0.086)
('parfaire cette édition', 0.0582)
('antérieure corne postérieure', 0.486)
('les cellules sont', 0.2331)
('ii canal central', 0.1677)
('16 et', 0.0078)
('fourni cet auteur', 0.0044)
('postérieure cordon postérieur', 0.5078)
('ces pièces proviennent', 0.0734)
('25 et', 0.0328)
('méthode de weigert', -0.0611)
('extrêmement intenses seules', -0.0653)
('avoisinante cellule ganglionnaire', 0.3836)
('la préparation précédente', 0.2169)
('rosés figure 30', 0.1267)
('voit la diminution', 0.1194)
('même sujet planche', 0.0422)
('faisceau pyramidal intact', 0.3281)
('38'

# PatternRank
* `keybert` + `keyphrase-vectorizers` = PatternRank<br>
 ❇️ pas besoin de spécifier la longueur des n-grammes à extraire, car la librairie l'infère elle-même<br>
❇️ la grammaticalité des phrases est prise en compte grâce aux extractions des parties du discours (p. ex. `<N.*>*<ADJ.*>*<ADJ.*>+`--> _sclérose latérale amyotrophique_)
* _cf._ [Schopf _et al._, 2022](https://arxiv.org/pdf/2210.05245.pdf) et [Schopf, 2022](https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db)



In [7]:
# !pip install keyphrase-vectorizers
# !pip install keybert
# !pip install flair
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
import os
from google.colab import drive
# Monter le Google Drive
drive.mount('/content/drive')

# Ajuster les chemins
path = '/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/corpus/'
input_file_name = 'test.txt'
output_file_name = '../output/test_output.txt'

# Initialiser le modèle KeyBERT multilingue
kw_model = KeyBERT(model=TransformerDocumentEmbeddings('google-bert/bert-base-multilingual-cased'))

# Ajuster les paramètres
vectorizer = KeyphraseCountVectorizer(spacy_pipeline='fr_core_news_lg', pos_pattern='<N.*>+<ADJ.*>*', stop_words='french')

with open(os.path.join(path, input_file_name), 'r') as input_file, \
     open(os.path.join(path, output_file_name), 'w') as output_file:
    raw_data = input_file.readlines()
    start = 0
    end = 22 # diviser le texte en tranches
    while start < len(raw_data):  # s'assurer que l'on traite toutes les données
        data = " ".join(raw_data[start:end]).replace('\n', ' ')  # Joindre les lignes and gérer les nouvelles lignes
        start = end
        end += 22
        try:
            # extraire les phrases-clés
            kp = kw_model.extract_keywords(data, vectorizer=vectorizer)
            for k in kp:
                print(k)
                output_file.write(str(k) + '\n')
        except ValueError as e:
            print(f"An error occurred while processing chunks starting at line {start}: {e}")
            # Accessoirement, écrire un message ou gérer l'erreur si besoin
            # output_file.write("Pas de phrases-clés extraites pour cette tranche.\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
('14 planches', 0.8301)
('progrès médical', 0.787)
('leçons', 0.7721)
('oeuvres complètes', 0.7623)
('35 figures', 0.7263)
('œuvres complè', 0.9246)
('réimpression', 0.9131)
('pachyméningite spinale', 0.9117)
('preuve convaincante', 0.8918)
('édition précédente', 0.89)
